In [12]:
import os
import time
import math
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
chain = Net.POLYGON
platform = Platform.SUSHI
contract = JSONContract.UniswapV2Pair

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [3]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoading().get_contract(web3, abi_path)
transfer_filt = Filter.create_filter(address=None, event_types=[Pair.events.Transfer])  # Listen events from any smart contract

In [4]:
reorg_mon = JSONRPCReorganisationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [5]:
start = start - 10

In [6]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=transfer_filt)

In [14]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['token0'] = Conversion().convert_uint256_hex_string_to_address(topics[1])
    event['details']['token1'] = Conversion().convert_uint256_hex_string_to_address(topics[2])

    if(len(arguments) >= 1):
        event['details']['transfer_value'] = Conversion().convert_int256_bytes_to_int(arguments[0])
    else:
        event['details']['transfer_value'] = math.nan
     
    events[k] = event
    if key not in processed_events:
        print(f"Transfer at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Transfer at block:60,311,198 tx:0x1fb0d6a4f2248df04efc8e1a3b0e921fbaf104de1545da574710e83caf00ee96
Transfer at block:60,311,198 tx:0x1fb0d6a4f2248df04efc8e1a3b0e921fbaf104de1545da574710e83caf00ee96
Transfer at block:60,311,198 tx:0x1fb0d6a4f2248df04efc8e1a3b0e921fbaf104de1545da574710e83caf00ee96
Transfer at block:60,311,198 tx:0x1fb0d6a4f2248df04efc8e1a3b0e921fbaf104de1545da574710e83caf00ee96
Transfer at block:60,311,198 tx:0x81f827b125c97baa2848ab29232cb019514eee78b7a369f045f34d25f69ac504
Transfer at block:60,311,198 tx:0x0e7266c0c28af898736a3eeb0adc2f81f9f00403c4b9f8fa7099c1d90d954086
Transfer at block:60,311,198 tx:0xdfcd31345b5638885a2cf0b222dbadfb1740592a867f927ea83f51829cda3509
Transfer at block:60,311,198 tx:0xb359aa2b07d6e969efe5843312261a655af7f0b9a585b3931a17c56486976f90
Transfer at block:60,311,198 tx:0xb359aa2b07d6e969efe5843312261a655af7f0b9a585b3931a17c56486976f90
Transfer at block:60,311,198 tx:0xc94d7de51ff91d6b595b9981bff351d6fb8627b449cb760bb264ed51c8df8474
Transfer a

In [15]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv2pair,transfer,sushi,0x3c499c542cef5e3811e1192ce70d8cc03d5c3359,0x1fb0d6a4f2248df04efc8e1a3b0e921fbaf104de1545...,60311198,1723058698,{'web3_type': {'address': '0x3c499c542cef5e381...
1,polygon,uniswapv2pair,transfer,sushi,0x251be3a17af4892035c37ebf5890f4a4d889dcad,0x1fb0d6a4f2248df04efc8e1a3b0e921fbaf104de1545...,60311198,1723058698,{'web3_type': {'address': '0x251be3a17af489203...
2,polygon,uniswapv2pair,transfer,sushi,0x3c499c542cef5e3811e1192ce70d8cc03d5c3359,0x1fb0d6a4f2248df04efc8e1a3b0e921fbaf104de1545...,60311198,1723058698,{'web3_type': {'address': '0x3c499c542cef5e381...
3,polygon,uniswapv2pair,transfer,sushi,0x3c499c542cef5e3811e1192ce70d8cc03d5c3359,0x1fb0d6a4f2248df04efc8e1a3b0e921fbaf104de1545...,60311198,1723058698,{'web3_type': {'address': '0x3c499c542cef5e381...
4,polygon,uniswapv2pair,transfer,sushi,0xc2132d05d31c914a87c6611c10748aeb04b58e8f,0x81f827b125c97baa2848ab29232cb019514eee78b7a3...,60311198,1723058698,{'web3_type': {'address': '0xc2132d05d31c914a8...
...,...,...,...,...,...,...,...,...,...
3154,polygon,uniswapv2pair,transfer,sushi,0x1195cf65f83b3a5768f3c496d3a05ad6412c64b7,0x187078bfc07b210e39b59bef35bdc4ad45c25b578eb2...,60311208,1723058720,{'web3_type': {'address': '0x1195cf65f83b3a576...
3155,polygon,uniswapv2pair,transfer,sushi,0xb2435253c71fca27be41206eb2793e44e1df6b6d,0x740761579091853a9fce3f301064a7115a3e3a373b05...,60311208,1723058720,{'web3_type': {'address': '0xb2435253c71fca27b...
3156,polygon,uniswapv2pair,transfer,sushi,0xe06bd4f5aac8d0aa337d13ec88db6defc6eaeefe,0x740761579091853a9fce3f301064a7115a3e3a373b05...,60311208,1723058720,{'web3_type': {'address': '0xe06bd4f5aac8d0aa3...
3157,polygon,uniswapv2pair,transfer,sushi,0xe06bd4f5aac8d0aa337d13ec88db6defc6eaeefe,0x740761579091853a9fce3f301064a7115a3e3a373b05...,60311208,1723058720,{'web3_type': {'address': '0xe06bd4f5aac8d0aa3...
